In [1]:
import warnings
warnings.filterwarnings('ignore')

# 1. Leer los datos de table

In [2]:
from bs4 import BeautifulSoup

with open('Table.html', 'r', encoding='utf-8') as file:
    html = file.read()

# Crear un objeto BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [3]:
import re

# Lista para guardar resultados
result = []

# Variables para rastrear la jerarquía actual
current_periodo_parlamentario = None
current_periodo_anual = None
current_legislatura = None

# Función para limpiar espacios adicionales
def clean_text(text):
    # Reemplazar múltiples espacios con uno solo
    return re.sub(r'\s+', ' ', text).strip()

# Recorrer todas las filas de la tabla
for tr in soup.find_all('tr', valign='top'):
    # Verificar si contiene un Periodo Parlamentario
    font_periodo_parlamentario = tr.find('font', string=lambda text: text and 'Congreso de la República' in text)
    if font_periodo_parlamentario:
        current_periodo_parlamentario = clean_text(font_periodo_parlamentario.text)

    # Verificar si contiene un Periodo Anual de Sesiones
    font_periodo_anual = tr.find('font', string=lambda text: text and 'Período Anual de Sesiones' in text)
    if font_periodo_anual:
        current_periodo_anual = clean_text(font_periodo_anual.text)

    # Verificar si contiene una Legislatura
    font_legislatura = tr.find('font', string=lambda text: text and 'Legislatura' in text)
    if font_legislatura:
        current_legislatura = clean_text(font_legislatura.text)

    # Buscar todos los enlaces en la fila
    a_tags = tr.find_all('a', href=True)
    
    # Procesar solo si hay enlaces
    if a_tags:
        # Buscar el enlace que tenga texto (no vacío)
        link_text = None
        link = None
        
        # Si el enlace tiene texto
        for a_tag in a_tags:
            text = clean_text(a_tag.get_text())
            if text:
                link = a_tag['href']
                link_text = text
                break
        
        # Si encontramos un enlace con texto, guardarlo
        if link and link_text:
            result.append({
                'periodo_parlamentario': current_periodo_parlamentario,
                'periodo_anual': current_periodo_anual,
                'legislatura': current_legislatura,
                'descripcion': link_text,
                'link': link
            })

# Mostrar los resultados
for item in result[13:15]:
    print(f"Periodo Parlamentario: {item['periodo_parlamentario']}")
    print(f"Periodo Anual de Sesiones: {item['periodo_anual']}")
    print(f"Legislatura: {item['legislatura']}")
    print(f"Descripción: {item['descripcion']}")
    print(f"Link: {item['link']}")
    print("-" * 40)

print("Total de resultados:", len(result))

Periodo Parlamentario: Congreso de la República - Periodo Parlamentario 2021 - 2026
Periodo Anual de Sesiones: Período Anual de Sesiones 2025 - 2026
Legislatura: Primera Legislatura Ordinaria
Descripción: Asistencias y votaciones de la sesión del 10-9-2025
Link: javascript:openWindow('Apleno/864A0C3F7EE3505705258D010079B963/$FILE/Asis_y_vot_del_10-9-2025.pdf')
----------------------------------------
Periodo Parlamentario: Congreso de la República - Periodo Parlamentario 2021 - 2026
Periodo Anual de Sesiones: Período Anual de Sesiones 2025 - 2026
Legislatura: Primera Legislatura Ordinaria
Descripción: Asistencias y votaciones de la sesión del 4-9-2025
Link: javascript:openWindow('Apleno/7842EE7A61C8330405258CFB005A6739/$FILE/Asis_y_vot_del_4-9-2025.pdf')
----------------------------------------
Total de resultados: 937


In [4]:
# Debug de los datos de los links
from collections import Counter

link_types = Counter()
for item in result:
    if 'javascript:openWindow(' in item['link']:
        link_types['javascript'] += 1
    else:
        link_types['otros'] += 1

print(link_types)

Counter({'javascript': 937})


# 2. Generar los links de descarga y nombre de cada archivo

In [5]:
import unicodedata
import re
import pandas as pd
import uuid

def clean_filename(text):
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ASCII', 'ignore').decode('utf-8')
    text = re.sub(r'[^\w\s-]', '', text)
    text = re.sub(r'\s+', '_', text)
    return text.strip().lower()

def generate_uuid_filename(url):
    """Genera un nombre de archivo único basado en la URL usando UUID5"""
    full_uuid = uuid.uuid5(uuid.NAMESPACE_URL, url)
    return f"{str(full_uuid)}.pdf"

# Filtrar los links válidos
clean_results = [item for item in result if 'javascript:openWindow(' in item['link']]

# Crear DataFrame
df_result = pd.DataFrame(clean_results)

# Extraer el link limpio
base_link = 'https://www2.congreso.gob.pe/Sicr/RelatAgenda/PlenoComiPerm20112016.nsf/'
df_result['clean_link'] = df_result['link'].str.extract(r"javascript:openWindow\('([^']+)'\)")
df_result['clean_link'] = base_link + df_result['clean_link']
df_result.drop('link', axis=1, inplace=True)

# Generar nombre de archivo único basado en clean_link
df_result['file_name'] = df_result['clean_link'].apply(generate_uuid_filename)

# Seleccionar columnas finales (AGREGADA 'file_name')
df_result = df_result[['periodo_parlamentario', 'periodo_anual', 'legislatura', 'descripcion', 'clean_link', 'file_name']]

# Guardar en archivo .txt con separador ";" y codificación utf-8
df_result.to_csv(
    'historico_documentos.csv',
    sep=',',
    header=True,
    index=False,
    encoding='utf-8'
)
df_result

,periodo_parlamentario,periodo_anual,legislatura,descripcion,clean_link,file_name
0,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2025 - 2026,Primera Legislatura Ordinaria,Asistencias y votaciones PROVISIONALES de la s...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,a90ab077-a704-539b-96fc-d6fc91df3dbd.pdf
1,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2025 - 2026,Primera Legislatura Ordinaria,Asistencias y votación de la sesión del 22-10-...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,cf308459-8db8-5e34-9352-4c63b559b756.pdf
2,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2025 - 2026,Primera Legislatura Ordinaria,Asistencias y votación de la sesión del 16-10-...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,ae5c634a-26b6-5ac8-9544-60ae0df44e61.pdf
3,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2025 - 2026,Primera Legislatura Ordinaria,Asistencias y votaciones de la sesión del 10-1...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,aaf8f003-a271-5ca0-a259-5ceca4de110b.pdf
4,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2025 - 2026,Primera Legislatura Ordinaria,Asistencias y votaciones de la sesión del 9-10...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,188f59f9-9657-5b53-9e09-5f876d8177f5.pdf
...,...,...,...,...,...,...
932,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2009 - 2010,Segunda Legislatura Ordinaria,Asistencia de la sesión del 15-06-2010,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,96b00db7-67a0-5d63-9540-da2fb930817d.pdf
933,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2009 - 2010,Segunda Legislatura Ordinaria,Asistencias y votaciones de la sesión del 10-0...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,cbe93d93-7e19-5b58-adc7-c0bea26fc446.pdf
934,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2009 - 2010,Segunda Legislatura Ordinaria,Asistencia de la sesión del 09-06-2010,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,3f7354f3-a8a3-5e4d-8b4d-9983081787f2.pdf
935,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2009 - 2010,Segunda Legislatura Ordinaria,Asistencias y votación de la sesión del 03-06-...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,a2c7dd77-91ce-5a27-aab6-23152e469b22.pdf


In [6]:
# Validar unicidad de combinación
combinaciones = df_result[['file_name','clean_link']]
if not combinaciones.duplicated().any():
    print("✅ Combinación única por fila.")
else:
    print("❌ Hay combinaciones duplicadas. Revisa los datos.")

✅ Combinación única por fila.
